# Baseline(TD-IDF + LR/XGB)  0.93
+ 未对文本做处理，直接提取TD-IDF特征，然后使用LR/XGBoost训练
+ 使用LR在验证集上的分数为0.918（20%作为验证集，训练时间4分钟）
+ 使用XGb在验证集上的分数为0.92（20%作为验证集,使用300维训练时间7分钟-16核）
+ 在使用XGB未调参的情况下最高F1-0.930，（使用全部六千多维特征，cpu 16核小半天跑完）

+ 之后可以尝试将高频词汇删除，在提取特征，并进行一些调参的工作

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm

C:\Users\jonxia\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [5]:
train = pd.read_csv('../../data/train_set.csv', sep='\t')
test = pd.read_csv('../../data/test_a.csv', sep='\t')

In [6]:
train_text = train['text']
test_text = test['text']
all_text = pd.concat([train_text, test_text])

In [7]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)
train_word_features

Wall time: 6min 50s


<200000x6977 sparse matrix of type '<class 'numpy.float64'>'
	with 56074040 stored elements in Compressed Sparse Row format>

# 逻辑回归

In [8]:
X_train = train_word_features
y_train = train['label']

# 可以改变输入维度
x_train_, x_valid_, y_train_, y_valid_ = train_test_split(X_train, y_train, test_size=0.2)
X_test = test_word_features

In [9]:
%%time
clf = LogisticRegression(C=4, n_jobs=16)
clf.fit(x_train_, y_train_)

y_pred = clf.predict(x_valid_)
train_scores = clf.score(x_train_, y_train_)
print(train_scores, f1_score(y_pred, y_valid_, average='macro'))

C:\Users\jonxia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jonxia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\jonxia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.94804375 0.9187810679581381
Wall time: 3min 22s


# XGB

In [10]:
class XGB():
    
    def __init__(self, X_df, y_df):
        self.X = X_df
        self.y = y_df
       
    def train(self, param):
        self.model = XGBClassifier(**param)
        self.model.fit(self.X, self.y, eval_set=[(self.X, self.y)],
                       eval_metric=['mlogloss'],
                       early_stopping_rounds=10,  # 连续N次分值不再优化则提前停止
                       verbose=False
                      )
        
#         mode evaluation
        train_result, train_proba = self.model.predict(self.X), self.model.predict_proba(self.X)
        train_acc = accuracy_score(self.y, train_result)
        print(self.y)
        print("**")
        print(train_proba)
        train_auc = f1_score(self.y, train_proba, average='macro')
        
        print("Train acc: %.2f%% Train auc: %.2f" % (train_acc*100.0, train_auc))
        
    def test(self, X_test, y_test):
        result, proba = self.model.predict(X_test), self.model.predict_proba(X_test)
        acc = accuracy_score(y_test, result)
        f1 = f1_score(y_test, proba, average='macro')
        
        print("acc: %.2f%% F1_score: %.2f%%" % (acc*100.0, f1))
    
    def grid(self, param_grid):
        self.param_grid = param_grid
        xgb_model = XGBClassifier(nthread=20)
        clf = GridSearchCV(xgb_model, self.param_grid, scoring='f1_macro', cv=2, verbose=1)
        clf.fit(self.X, self.y)
        print("Best score: %f using parms: %s" % (clf.best_score_, clf.best_params_))
        return clf.best_params_, clf.best_score_

    

In [11]:
x_train_, x_valid_, y_train_, y_valid_ = train_test_split(X_train[:, :300], y_train, test_size=0.2, shuffle=True, random_state=42)
X_test = test_word_features[:,:300]

In [12]:
%%time
param = {'learning_rate': 0.05,         #  (xgb’s “eta”)
              'objective': 'multi:softmax', 
              'n_jobs': 16,
              'n_estimators': 300,           # 树的个数
              'max_depth': 10,               
              'gamma': 0.5,                  # 惩罚项中叶子结点个数前的参数，Increasing this value will make model more conservative.
              'reg_alpha': 0,               # L1 regularization term on weights.Increasing this value will make model more conservative.
              'reg_lambda': 2,              # L2 regularization term on weights.Increasing this value will make model more conservative.
              'min_child_weight' : 1,      # 叶子节点最小权重
              'subsample':0.8,             # 随机选择80%样本建立决策树
              'random_state':1           # 随机数
             }
model = XGB(x_train_, y_train_)
model.train(param)
model.test(x_valid_, y_valid_)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
final_model = XGB(X_train, y_train)
final_model.train(param)

submission = pd.read_csv('./datalab/72510/test_a_sample_submit.csv')
preds = final_model.model.predict(X_test)
submission['label'] = preds
submission.to_csv('./xgb_submission.csv', index=False)